## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$


In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
# 학습 데이터를 불러 옵니다.
df_train = pd.read_csv('train_prob.csv', index_col='id')
# targetA를 예측하는 모델을 활용하기 위해 가져옵니다.
df_train['targetA'] = df_train['target'] <= 7.45

In [3]:
# 테스트 데이터를 불러 옵니다.
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [4]:
# 전처리를 가져왔습니다.
repl_list= [
    ('cat3', {'B': 'C'}, [83634, 147361, 9005]),
    ('cat4', {'A': 'B', 'D': 'B'}, [239397, 603]),
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]),
]

for c, d, cnt in repl_list:
    df_train[c] = df_train[c].replace(d)
    #df_test[c] = df_test[c].replace(d) 일부러 아래 고찰을 위해 주석처리합니다.

## OneHotEncoder 시  Unknown 범주값 대처법에 대한 고찰

Unknown 범주값: Test 셋에서 Train에 나오지 않는 범주값을 일컫습니다.

Unknown 범주값에 대해서 OneHotEncoder 에서 handle_unknown='ignore'로 하면,

그 변수에 대해서는 모든 값이 0이 가변수를 반환합니다.

하지만, OneHotEncoder를 handle_unknown='ignore'로 하면 drop='first'를 사용할 수 없습니다.

(1.0 이후 버젼은 허용합니다. 쩝...)

선형 모델을 사용하지 않으면 문제 없습니다. 

그런데, 선형 모델을 사용한다면 drop='first'를 사용하지 않으면, 가변수 간에 완전한 다중공선성을 갖게 됩니다.

이렇게 되면, 선형 모델의 계수(coefficient)가 큰 값을 지니게 됩니다. 

따라서, 불안정한 모델이 만들어지게 되고, 

학습에서 등장하지 않은 경우(Ex 가변수가 모두 0)에 대해서는 전혀 엉뚱한 값이 나오게 됩니다.

선형 모델을 사용하고자 하고, 사용하고자 하는 변수가 그렇다면.

이에 대한 대처법 test에 train에 나오지 않는 범주값이 나올 경우 대체 시켜줍니다. 

   Ex) 가장 많이 나온 범주


In [5]:
# test에 등장하지 않은 내용이 있는지 검사하는 로직입니다. ㅠ.ㅜ 
# 선형 모델을 사용하겠다면, 출력된 값들은 어찌 됐는 처리가 되야겠죠...
df_test.select_dtypes('object').apply(
    lambda x: set(x.unique()) - set(df_train[x.name].unique())
)

cat0                 {}
cat1                 {}
cat2                 {}
cat3                {B}
cat4             {A, D}
cat5                 {}
cat6    {E, H, G, D, I}
cat7          {F, A, C}
cat8             {F, B}
cat9          {E, D, C}
dtype: object

In [6]:
# 요런 식으로 말이죠. 이렇게 해야지 handle_unknown='ignore'를 쓰지 않을 수 있게 되어, drop='first'를 사용할 수 있습니다.
for c, d, cnt in repl_list:
    df_test[c] = df_test[c].replace(d)
df_test.select_dtypes('object').apply(
    lambda x: set(x.unique()) - set(df_train[x.name].unique())
)

cat0    {}
cat1    {}
cat2    {}
cat3    {}
cat4    {}
cat5    {}
cat6    {}
cat7    {}
cat8    {}
cat9    {}
dtype: object

## OneHotEncoder시 Unknown 범주값 대처법에 대한 고찰에 대한 마무리 

**선형 모델(LinearRegression, LogisticRegression, ...)**을 사용을 한다면 test에 train에 등장하지 않은 것들이 없게 처리를 해주고,

OneHotEncoder를 사용시 drop='first'를 사용하시어 안정적인 모델을 만드는 것을 권유 드립니다.

In [7]:
# 문제3의 내용을 응용합니다.
from scipy.stats import norm
mu_A, std_A = 6.769, 0.616
mu_B, std_B = 8.123, 0.527
df_train_A = df_train.assign(
    prob_A = 1 - norm.cdf(df_train['target'], loc=mu_A, scale=std_A), # A의 분포에서, 우측 꼬리에 해당하는 영역을 구합니다.
    prob_B = norm.cdf(df_train['target'], loc=mu_B, scale=std_B) # B의 분포에서, 좌측 꼬리에 해당하는 영역을 구합니다.
)
df_train_A = df_train_A.query('prob_B < 0.01 or prob_A < 0.01').copy()
df_train_A.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont8,cont9,cont10,cont11,cont12,cont13,target,targetA,prob_A,prob_B
id,,,,,,,,,,,,,,,,,,,,,
29503,A,B,A,C,B,B,A,E,E,F,...,0.30163,0.32551,0.26982,0.19329,0.39034,0.30002,6.66255,True,0.568599,2.792056e-03
113326,A,B,A,A,B,D,A,E,C,H,...,0.65004,0.63958,0.60535,0.70951,0.71664,0.78782,5.98141,True,0.899473,2.414650e-05
199507,A,B,B,A,B,B,A,E,A,L,...,0.22571,0.48717,0.33999,0.51461,0.37367,0.69462,6.60870,True,0.602656,2.030176e-03
346281,A,A,A,D,B,D,A,E,C,F,...,0.45999,0.79100,0.62269,0.83624,0.78785,0.30893,5.42339,True,0.985535,1.506759e-07
137333,A,A,A,C,B,D,A,D,C,K,...,0.49416,0.89742,0.59422,0.72298,0.72535,0.82115,8.34177,False,0.005337,6.609742e-01


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

In [9]:
import xgboost as xgb
X_cont = ['cont{}'.format(i) for i in range(14)]
X_cat = ['cat{}'.format(i) for i in range(10)]
X_xgb = X_cont + X_cat
ct_xgb = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), X_cat),
    ('pt', 'passthrough', X_cont)
])
clf_xgb = xgb.XGBClassifier(
    max_depth=2, #트리의 최대 깊이: 2
    reg_alpha=0.1, # L1 규제: 0.1
    reg_lambda=0.1, # L2 규제: 0.1
    colsample_bytree=0.25, #트리 당 컬럼 샘플링 비율: 0.25
    n_estimators=500, #트리의 수: 500
    random_state=123
)
clf_xgb = make_pipeline(ct_xgb, clf_xgb)
clf_xgb.fit(df_train_A[X_xgb], df_train_A['targetA'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=False),
                                                  ['cat0', 'cat1', 'cat2',
                                   

In [10]:
# targetA 예측 확률을 파생 변수로 삼습니다.
df_train['target_probA'] = clf_xgb.predict_proba(df_train[X_xgb])[:, 1]
df_test['target_probA'] = clf_xgb.predict_proba(df_test[X_xgb])[:, 1]

In [11]:
# 문제 4에서 만든 파생변수를 거져 옵니다.
q = [i / 100 for i in range(101)] # np.linspace(0, 1, 101)
for i in range(0, 14):
    col = 'cont{}'.format(i)
    col_q = col + '_q'
    q_val = df_train[col].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_int = pd.cut(df_train[col], q_val)
    q_mean = df_train.groupby(q_int)['target'].mean()
    df_train[col_q] = q_int.map(q_mean).astype('float')
    df_test[col_q] = pd.cut(df_test[col], q_val).map(q_mean).astype('float')

In [12]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
cv = KFold(n_splits=5, random_state=123, shuffle=True)

In [13]:
# Baseline 모델을 만들어봅니다.
from sklearn.linear_model import LinearRegression

X_cont = ['cont{}_q'.format(i) for i in range(14)] + ['target_probA']
X_cat = ['cat{}'.format(i) for i in range(10)]
X_lr = X_cont + X_cat

ct_lr = ColumnTransformer([
    ('ohe', OneHotEncoder(sparse=False, drop='first'), X_cat),
    ('pt', 'passthrough', X_cont)
])

reg_lr = make_pipeline(ct_lr, LinearRegression())
scores_ = cross_val_score(reg_lr, df_train[X_lr], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_, np.mean(scores_)

(array([-0.70886461, -0.71123524, -0.71662955, -0.70661583, -0.70125227]),
 -0.708919501126037)

In [14]:
# Baseline 결과를 만듭니다.
reg_lr.fit(df_train[X_lr], df_train['target'])
pd.DataFrame(
    reg_lr.predict(df_test[X_lr]),
    columns=['target'],
    index=df_test.index
).to_csv('answer6.csv')

In [15]:
# 정답셋과 비교합니다.
mse = mean_squared_error(df_ans['target'], reg_lr.predict(df_test[X_lr]))
mse, mse ** 0.5 # RMSE로 뽑아옵니다 

(0.7188957818884829, 0.8478772209987027)

In [16]:
# XGBoost 모델로 해봅니다.
# 실행시간의 압박이 있습니다.
X_cont = ['cont{}'.format(i) for i in range(14)] + ['target_probA']
X_cat = ['cat{}'.format(i) for i in range(10)]
X_xgb = X_cont + X_cat
ct_xgb = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), X_cat),
    ('pt', 'passthrough', X_cont)
])

reg_xgb = xgb.XGBRegressor(
    max_depth=2, 
    random_state=123,
    colsample_bytree=0.25, 
    n_estimators=500
)

reg_xgb = make_pipeline(ct_xgb, reg_xgb)
scores_ = cross_val_score(reg_xgb, df_train[X_xgb], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_, np.mean(scores_)

(array([-0.71064094, -0.7128614 , -0.71806878, -0.7083828 , -0.70272486]),
 -0.7105357560846794)

In [17]:
reg_xgb.fit(df_train[X_xgb], df_train['target'])
# 정답셋과 비교합니다.
mse = mean_squared_error(df_ans['target'], reg_xgb.predict(df_test[X_xgb]))
mse, mse ** 0.5 # RMSE로 뽑아옵니다 

(0.717977620100122, 0.8473356006330207)

In [18]:
# 두 모델을 앙상블 해봅니다.
# 실행시간의 압박이 있습니다.
from sklearn.ensemble import VotingRegressor

X_vt = ['cont{}'.format(i) for i in range(14)] + ['cont{}_q'.format(i) for i in range(14)] + X_cat + ['target_probA']
reg_vt = VotingRegressor([
    ('lr', reg_lr),
    ('xgb', reg_xgb)
])
scores_ = cross_val_score(reg_vt, df_train[X_vt], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_, np.mean(scores_)

(array([-0.70876365, -0.71104412, -0.71636808, -0.70652537, -0.70100968]),
 -0.7087421803791012)

In [19]:
# 가장 좋은 모델을 최종 모델로 합니다.
# VotingRegressor가 가장 좋네요
reg_vt.fit(df_train[X_vt], df_train['target'])
pd.DataFrame(
    reg_vt.predict(df_test[X_vt]),
    columns=['target'],
    index=df_test.index
).to_csv('answer6.csv')

In [20]:
# 정답셋과 비교합니다. 아주 미묘하게 좋습니다.
mse = mean_squared_error(df_ans['target'], reg_vt.predict(df_test[X_vt]))
mse, mse ** 0.5 # RMSE로 뽑아옵니다.

(0.717490926802164, 0.8470483615485978)